In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, LinearRegression,RidgeCV
from sklearn.metrics import mean_squared_error

plt.style.use('ggplot')
%matplotlib inline

In [28]:
df=pd.read_csv('lasso.csv',index_col=0)

In [29]:
df.head()

,LotArea,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,SalePrice,BsmtScore,TotalBath,MSSubClass_90,...,KitchenQual,Functional,FireplaceQu,GarageCond,PavedDrive,PoolQC,Fence,MoSold_Autumn,MoSold_Spring,MoSold_Winter
Id,,,,,,,,,,,,,,,,,,,,,
1,8450,3,1,8,0,2,208500,5.992928,3.5,0,...,4,8,1,4,3,1,1,0,0,1
2,9600,3,1,6,1,2,181500,4.995914,2.5,0,...,3,8,4,4,3,1,1,0,1,0
3,11250,3,1,6,1,2,223500,5.989733,3.5,0,...,4,8,4,4,3,1,1,1,0,0
4,9550,3,1,7,1,3,140000,4.981567,2.0,0,...,4,8,5,4,3,1,1,0,0,1
5,14260,4,1,9,1,3,250000,5.992378,3.5,0,...,4,8,4,4,3,1,1,0,0,1


In [30]:
SalePrice=df.SalePrice
ridge_df=df.drop('SalePrice',axis=1)

# Split the Ames_df into a 80% train-set and a 20% test set.
np.random.seed(19)
testIdxes = np.random.choice(range(1458), size= 292, replace=False)
trainIdxes = list(set(range(1458))-set(testIdxes))
trainX = ridge_df.iloc[trainIdxes]
testX  = ridge_df.iloc[testIdxes]
trainY = SalePrice.iloc[trainIdxes]
testY  = SalePrice.iloc[testIdxes]

In [31]:
alphaRange = np.linspace(1e-3,200,40) # Use np.linspace to generate 40 alpha
ridge_cv = RidgeCV(alphas = alphaRange,cv=10) #cross validation to get best lambda
ridge_cv.fit(trainX, trainY)
best_alpha=ridge_cv.alpha_
best_alpha=round(best_alpha,3)
best_alpha

0.001

In [32]:
ridge=Ridge()
ridge.set_params(alpha=best_alpha)
ridge.fit(trainX,trainY)
ridge_intercept = ridge.intercept_
ridge_coef = pd.Series(ridge.coef_,index=ridge_df.columns)
rtrain_score=ridge.score(trainX,trainY)
rtest_score= ridge.score(testX,testY)

In [33]:
print ('intercept \t',ridge_intercept)
print ('train_score(R2)\t',rtrain_score)
print ('test_score(R2) \t',rtest_score)

intercept 	 -162524.663938285
train_score(R2)	 0.8873423684373635
test_score(R2) 	 0.8568726613345821


In [34]:
ridge_predict = ridge.predict(testX)
RMSE=np.sqrt(mean_squared_error(testY,ridge_predict))
RMSE

25777.59438425063

In [35]:
ridge.score(trainX,trainY)

0.8873423684373635

In [36]:
ridge.score(testX,testY)

0.8568726613345821

In [37]:
#Sorted the coefs, the larger the number is , the larger impact it has to the whole model
#showing the top 60 features
sorted_coef=abs(ridge_coef).sort_values(ascending=False)
sorted_coef[1:60]

Neighborhood_StoneBr     59219.621595
Neighborhood_NoRidge     59096.869087
Neighborhood_NridgHt     35989.619534
PoolQC                   32812.974600
Exterior_ImStucc         31978.594442
HouseStyle_SFoyer        31108.072334
BldgType_Twnhs           26901.123328
Neighborhood_Crawfor     25268.366540
HouseStyle_SLvl          20346.173603
Condition_RRAe           20261.221753
BldgType_TwnhsE          19990.335710
SaleCondition_Partial    19787.095519
MSSubClass_190           17988.861171
Exterior_BrkFace         17222.325247
Neighborhood_Gilbert     15717.048523
Foundation_Slab          14145.498940
Exterior_CemntBd         13919.118877
Alley_Pave               13068.388564
GarageType_BuiltIn       12745.423665
LotConfig_CulDSac        12591.583819
TotalBath                12571.084844
Fireplaces               12034.095780
OverallQual              11941.922775
GarageCars               11922.355649
TotRmsAbvGrd             11736.623526
MSSubClass_120           11698.114540
LotShape_IR2

In [38]:
PredictionsTest=ridge.predict(testX)
RMSLEvalue = np.sqrt(np.mean(np.power(np.log1p(testY)-np.log1p(PredictionsTest), 2)))
RMSLEvalue

0.1616418364373594